ETL User Reviews

In [34]:
#Importando librerias
import json
import numpy as np
import pandas as pd
import ast
import gzip
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

1. Extraccion de datos

In [5]:
# Definimos una función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(nombreArchivo):

    # Abrimos el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(nombreArchivo, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [6]:
reviews_1 = leer_json_gzip('data/user_reviews.json.gz')
reviews_2 = pd.DataFrame(reviews_1)
reviews_2.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


2. Exploración de datos

In [7]:
#Desanidar la variable reviews, contiene la información de los comentarios sobre los juegos
f=[]
for i in reviews_2.values:
    f.append(i[2])

reviews_3 = pd.DataFrame([review for sublist in f for review in sublist])

In [8]:
reviews_3.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


2.1 Unir la variable reviews desanidada nuevamente para tener vinculado el user_id

In [10]:
reviews_3.shape #registros de la tabla review desanidada

(59305, 7)

In [11]:
reviews_2.shape #registros de la tabla review sin desanidadar

(25799, 3)

Obs. Al desanidar reviews se genero una tabla con 59.305 registros y la tabla original tiene 25.799, por lo cual la tabla con user_id haremos un explode para equiparar ambas tablas y unirlas

In [12]:
reviews_4 = reviews_2.explode('reviews')
reviews_4.shape

(59333, 3)

In [13]:
reviews_4.head(3)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."


In [17]:
#reseteo el index
reviews_4.reset_index(inplace=True)
reviews_3.reset_index(inplace=True)

In [18]:
#unifico
reviews_5 = pd.concat([reviews_4,reviews_3], axis=1)
reviews_6 = reviews_5.drop(columns = ['reviews','index','funny','user_url','last_edited'])
reviews_6.head(3)

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


*****************************************************

2.2 Revision de duplicados

In [19]:
duplicados=reviews_6.loc[reviews_6.duplicated()]
duplicados=duplicados.sort_values('user_id')
duplicados.head(5)

,user_id,posted,item_id,helpful,recommend,review
25139,0-3-0,"Posted May 2, 2014.",244850,No ratings yet,True,Like playing minecraft in space very addictive...
8027,111222333444555666888,Posted February 1.,213670,No ratings yet,True,"Jogo muito bom e engraçado,especialmente para ..."
8028,111222333444555666888,Posted February 1.,730,No ratings yet,True,"Jogo muito viciante e variado...Enfim,recomend..."
8029,111222333444555666888,"Posted November 21, 2015.",307880,No ratings yet,True,Good game when you can make your house.Hahaha!
8030,111222333444555666888,"Posted November 6, 2015.",8870,No ratings yet,True,Esse jogo me impressionou.Comprei o triple pac...


Obs: Un mismo usuario comento sobre distintos item_id que seria distintos juegos, por lo cual es valido

2.3 Revision de nulos

In [20]:
nulos= reviews_6.isnull().sum()
nulos # son productos de la union entre ambas tablas

user_id       0
posted       28
item_id      28
helpful      28
recommend    28
review       28
dtype: int64

In [29]:
reviews_7 = reviews_6.dropna().reset_index(drop=True)

2.4 Transformación variable fecha

In [30]:
reviews_7['posted'] = reviews_7['posted'].str.extract(r'Posted([\w\s\d,]+)') #Reemplazo la palabra Posted por espacio vacio
reviews_7['year'] = reviews_7['posted'].str.extract(r'(\d{4})').fillna('0') #extraigo el año y relleno los nan por 0
reviews_8=reviews_7.drop(columns=['posted']) #elimino la columna

In [31]:
reviews_8.head(3)

,user_id,item_id,helpful,recommend,review,year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011


2.5 Al importar el Jason, muchas variables son object y lo queremos pasas a string/int dependiendo sea el caso

In [32]:
reviews_8.dtypes

user_id      object
item_id      object
helpful      object
recommend    object
review       object
year         object
dtype: object

In [33]:
reviews_8.apply(lambda x: pd.api.types.infer_dtype(x.values))

user_id       string
item_id       string
helpful       string
recommend    boolean
review        string
year          string
dtype: object

2.6 Procedemos al analisis de sentimientos

Se debe crear la columna ***'sentiment_analysis'*** aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de `1`.

Obs. La función sentiment nos devuelve un objeto Sentiment que contiene la polaridad y subjetividad del texto. La polaridad varía entre -1 y 1, donde -1 indica una emocion muy negativa y 1 una emoción muy positiva. Si el número es cero, significa que el texto es neutro.
La subjetividad variará entre 0 y 1, donde 0 indica que el texto es muy objetivo y 1 muy subjetivo.

In [35]:
def analizar_sentimiento(variable):
    if variable is None:
        return 1 #Neutral
    analisis = TextBlob(variable)
    polaridad = analisis.sentiment.polarity
    if polaridad >= 0.25:
        return 2 #Positivo
    elif polaridad <= -0.33:
        return 0 #Negativo
    else:
        return 1 #Neutral
    
#si la polaridad va de 1 a -1, siendo 1 positivo, 0 neutro y -1 negativo, hay que realizar una partición eficiente
#para que no haya un desbalance en la distribución de la nueva variable, por lo cual siendo 0.33, la distancia entre
#cada nodo es entre 0.66/0.67

#luego de ciertas validaciones, comentarions como  best zombie game i have played, daba un 1, asi que no seria neutral

In [36]:
reviews_8["sentiment_analisis"]= reviews_8["review"].astype(str).apply(analizar_sentimiento)
reviews_8.head(3)

,user_id,item_id,helpful,recommend,review,year,sentiment_analisis
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,1


Adjunto su polaridad tambien para ver que tan bien se aplica a los comentarios

In [37]:
def sentiment_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None

reviews_8['polaridad'] = reviews_8['review'].apply(sentiment_calc)

In [38]:
reviews_8.head(5)

,user_id,item_id,helpful,recommend,review,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2,0.337500
2,76561197970982479,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,1,0.050000
3,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,1,0.114583
4,js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,1,-0.036111


Realizamos controles al azar

In [40]:
reviews_8["review"][15]

'It was a great game from what I played, right now I need to find the actual download.'

In [41]:
reviews_8["sentiment_analisis"][15]

1

In [42]:
reviews_8["review"][1183]

'best zombie game i have played'

In [43]:
reviews_8["sentiment_analisis"][1183]

2

2.7 Eliminar columnas que no necesitamos

In [44]:
reviews_9 = reviews_8.drop(columns=["review"])
reviews_9.head(2)

,user_id,item_id,helpful,recommend,year,sentiment_analisis,polaridad
0,76561197970982479,1250,No ratings yet,True,2011,1,0.174444
1,76561197970982479,22200,No ratings yet,True,2011,2,0.337500


2.8 Guardo la tabla para reutilizarla más adelante

In [45]:
reviews_9_csv = "data/reviews_9.csv"
reviews_9.to_csv(reviews_9_csv , index=False, encoding="utf-8")